In [25]:
#with intensities THIS BOX IS A MERGER OF THE FOLLOWING 3 BOXES TO SPEED UP PROCESSING
import itertools
from itertools import product
import pandas as pd
from os import *

def make_patterns(s):                                     #This  funkction generates all possible peptide sequences due to the indistinguishability of leucine and isoleucine. The input is a string of an amino acid sequence, and the output is a generator that yields all possible sequences resulting from replacing the "IL" amino acids in the input string with either "I" or "L".

    IndistinguishableAminoAcids = 'IL'

                                                            # Convert input amin oacid sequence into a list so we can easily substitute single amino acids
    seq = list(s)

                                                            # Find indices of IndistinguishableAminoAcids in seq
    indices = [i for i, c in enumerate(seq) if c in IndistinguishableAminoAcids]

                                                            # Generate combinations of IndistinguishableAminoAcids & place them into the list
    for t in product(IndistinguishableAminoAcids, repeat=len(indices)):
        for i, c in zip(indices, t):
            seq[i] = c
        yield ''.join(seq)
        
def most_frequent(List):                                  # This function takes a list as an input and returns the most frequent value in the list.
    return max(set(List), key = List.count)               # Use the max function and the count method to determine the most frequent value in the list. The key argument specifies that the max function should use the count method to determine the maximum.

def most_frequent_frequency_proportion(list_data):       # This function takes a list as an input and returns the proportion of the frequency of the most frequent value in the list.
    stat_frequency = {}                                   # Initialize an empty dictionary to store the frequency of each value in the list.
    stat_proportion = {}                                 # Initialize another empty dictionary to store the proportion of each frequency.
    total = len(list_data)                                # Calculate the total number of elements in the list.
    for e in list_data:                                   # Loop through each element in the list.
        if str(e) in stat_frequency:                      # If the current element is already in the frequency dictionary, increment its frequency.
            stat_frequency[str(e)] += 1
        else:                                             # If the current element is not in the frequency dictionary, add it and set its frequency to 1.
            stat_frequency[str(e)] = 1
    for key, value in stat_frequency.items():             # Loop through each key-value pair in the frequency dictionary.
        stat_proportion[key] = value / total             # Calculate the proportion of each frequency by dividing it by the total number of elements in the list.
    return max(stat_proportion.values())                 # Return the maximum proportion of frequency, which represents the proportion of the most frequent value in the list.


def remove_duplicates_from_list(x):                       #This function removes duplicates from a list
    return list(dict.fromkeys(x))

def get_duplicates_from_list(L):                          #This function returns duplicate values from a list
    seen = set()
    seen2 = set()
    seen_add = seen.add
    seen2_add = seen2.add
    for item in L:
        if item in seen:
            seen2_add(item)
        else:
            seen_add(item)
    return list(seen2)

def get_intersection_from_two_lists(x,y):               #This function returns the intersecition of two lists
    return list(set(x).intersection(y))


# load the trypsin-digest tsv of target database and contaminant database in the respective dataframe
input_filename = "C:\\Users\\klaes\\Nextcloud\\Shared with me\\Simon_Myriel_Lorenz\\WP1_GroEL_method\\Python\\testfiles\\clean_noArchaea_clean_GroEL_290780_trypsin_digests_plus_LPSN_Taxonomy_2missedcleavage_6to144aa+AK199+CV026.tsv"
contaminants_filename = "C:\\Users\\klaes\\Nextcloud\\Shared with me\\Simon_Myriel_Lorenz\\WP1_GroEL_method\\Python\\testfiles\\crap_2missedcleavage_6to144aa.tsv"

print(input_filename)
df_target_database=pd.read_csv(input_filename, sep = '\t')
df_contaminants=pd.read_csv(contaminants_filename, sep = '\t')

#create a set (non redundant) of all tryptics peptides in the target database and contaminant database, respectively
all_tryp_peps_set = set(itertools.chain.from_iterable(df_target_database.TrypPeps.apply(lambda x: x[2:-2].split("', '"))))
all_contaminant_peps_set = set(itertools.chain.from_iterable(df_contaminants.TrypPeps.apply(lambda x: x[2:-2].split("', '"))))

#convert the tryptic peptides column to lists to facilitate counting of hits
df_target_database['TrypPeps'] = df_target_database.TrypPeps.apply(lambda x: x[2:-2].split("', '"))
df_contaminants['TrypPeps'] = df_contaminants.TrypPeps.apply(lambda x: x[2:-2].split("', '"))



#WITHOUT TECHNICAL REPLICATES
#MetaProSIP Output files are loaded subsequently and peptide sequences are extracted to a list "found_seqs". Modifications (Carbamidomythylation and Oxidation) are being removed
for o in range(8):
    df_database = df_target_database
    df_results = []
    found_seqs = []
    MetaProSIP_Output_filename = "C:\\Users\\klaes\\Nextcloud\\Cloud\\WP1\\Reference Data\\5ppm\\TP_biculture\\Galaxy"+str(179+(2*o))+"-[MetaProSIP_on_data_"+str(139+o)+",_data_22,_and_data_"+str(23+o)+"__out_peptide_centric_csv].csv"
    print(MetaProSIP_Output_filename)
    MetaProSIP_Output = pd.read_csv(MetaProSIP_Output_filename, sep = '\t')
    sample_name = str(MetaProSIP_Output.iloc[0,4]).split('/')[1].split('.')[0]
    python_output_filename = "C:\\Users\\klaes\\Nextcloud\\Cloud\\WP1\\Reference Data\\5ppm\\TP_biculture\\analyzed\\"+sample_name+"_test.tsv"
    python_intermediate_filename = "C:\\Users\\klaes\\Nextcloud\\Cloud\\WP1\\Reference Data\\5ppm\\TP_biculture\\analyzed\\"+sample_name+"_intermediate_test.tsv"
    python_contaminants_output_filename = "C:\\Users\\klaes\\Nextcloud\\Cloud\\WP1\\Reference Data\\5ppm\\TP_biculture\\analyzed\\"+sample_name+"_contaminants_test.tsv"
    found_seqs = MetaProSIP_Output['Peptide Sequence'].tolist()
    found_seqs = [str(x).replace('(Carbamidomethyl)', '').replace('(Oxidation)', '') for x in found_seqs] #.replace('.(Acetyl)', '') only for Eukarya
    
    print(found_seqs)
    print(python_output_filename)

#WITH TECHNICAL DUPLICATES:
#for o in range(4):
#    d7=[]
#    MetaProSIP_Output_filename1="C:\\Users\\klaes\\Nextcloud\\Cloud\\WP1\\Reference Data\\equal protein mass\\MetaProSIP_Exports\\Galaxy"+str(88+(2*o))+"-[MetaProSIP_on_data_"+str(68+o)+",_data_11,_and_data_"+str(12+o)+"__out_peptide_centric_csv].csv"
#    MetaProSIP_Output_filename2="C:\\Users\\klaes\\Nextcloud\\Cloud\\WP1\\Reference Data\\equal protein mass\\MetaProSIP_Exports\\Galaxy"+str(96+(2*o))+"-[MetaProSIP_on_data_"+str(72+o)+",_data_11,_and_data_"+str(16+o)+"__out_peptide_centric_csv].csv"
#    print(MetaProSIP_Output_filename1 + MetaProSIP_Output_filename2)
#    MetaProSIP_Output1=pd.read_csv(MetaProSIP_Output_filename1, sep = '\t')
#    MetaProSIP_Output2=pd.read_csv(MetaProSIP_Output_filename2, sep = '\t')
#    sample_name = str(MetaProSIP_Output1.iloc[0,4]).split('/')[1].split('.')[0]+str(MetaProSIP_Output2.iloc[0,4]).split('/')[1].split('.')[0]
#    python_output_filename="C:\\Users\\klaes\\Nextcloud\\Cloud\\WP1\\Reference Data\\equal protein mass\\MetaProSIP_Exports\\analyzed\\"+sample_name+".csv"
#    python_intermediate_filename="C:\\Users\\klaes\\Nextcloud\\Cloud\\WP1\\Reference Data\\equal protein mass\\MetaProSIP_Exports\\analyzed\\"+sample_name+"_intermediate.csv"
#    found_seqs= pd.concat([MetaProSIP_Output1, MetaProSIP_Output2], axis=0)['Peptide Sequence'].tolist()
#    found_seqs= [str(x).replace('(Carbamidomethyl)', '').replace('(Oxidation)', '') for x in found_seqs]
#    print(found_seqs)
#    print(python_output_filename)
    
    #found seqs_wobbled is a list with all detected peptides and also their isobaric equivalents (non-redundant)   
    found_seqs_wobbled=[]
    found_contaminant_peps_wobbled=[]

    #The found_seqs list is extended to found_seqs_wobbled by adding the isobaric equivalent of the detected peptides if it is also in the database. Peptides that are also in the contaminant database are excluded from found_seqs_wobbled    
    for i in found_seqs:
        for s in make_patterns(i):
            if s in all_contaminant_peps_set:
                if s not in found_contaminant_peps_wobbled:
                    found_contaminant_peps_wobbled.append(s)
            if s in all_tryp_peps_set:
                if s not in all_contaminant_peps_set:
                    if s not in found_seqs_wobbled:
                        found_seqs_wobbled.append(s)
            
    # set the counts to 0 before start counting and reset the following dataframes
    df_database.loc[:,"HitPepSeq"] = str("")
    df_database.loc[:,"Count"] = 0 
    df_HPSM = pd.DataFrame()
    df_results = pd.DataFrame()
    df_results_min5_family = pd.DataFrame()
    df_results_min5 = pd.DataFrame()
    total_hit_count = 0
    df_contaminants.loc[:,"HitPepSeq"] = str("")
    df_contaminants.loc[:,"Count"] = 0
    total_hit_count_contaminants = 0
    taxonomic_columns = ['Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus']

    
    # loop through contaminant peptides
    for contaminant_peptide in found_contaminant_peps_wobbled:
        # List with True/False - true if peptide is in contaminant database
        mask=[contaminant_peptide in x for x in df_contaminants['TrypPeps']] 
        # apply mask: for all proteins in the contaminant database that contain this tryptic peptide (all rows in which mask is true), "count" column is increased by 1 and the peptide is added to "HitPepSeq" column
        df_contaminants.loc[mask,"Count"] += 1
        df_contaminants.loc[mask,"HitPepSeq"] = df_contaminants.loc[mask,"HitPepSeq"] + contaminant_peptide + str(",")
        # print how often each peptide was found in the contaminant database
        print(contaminant_peptide, found_contaminant_peps_wobbled.index(contaminant_peptide), "/", len(found_contaminant_peps_wobbled), "No. of hits in the contaminant database:", len(df_contaminants[mask]))
        total_hit_count_contaminants+=len(df_contaminants[mask])
    # print how often contaminant peptides were found in total in the contaminant database
    print("Total found hits over all contaminant peptides:", total_hit_count_contaminants)
    df_contaminants['HitPepSeq']=df_contaminants.HitPepSeq.apply(lambda x: x.split(","))
    #save contaminants dataframe as tsv-file
    df_contaminants.to_csv(python_contaminants_output_filename, sep = '\t')

    # loop through the detected GroEL peptides
    for peptide in found_seqs_wobbled:
        # List with True/False - true if peptide is in GroEL database
        mask=[peptide in x for x in df_database['TrypPeps']] 
        # apply mask: for all proteins in the GroEL database that contain this tryptic peptide (all rows in which mask is true), "count" column is increased by 1 and the peptide is added to "HitPepSeq" column
        df_database.loc[mask,"Count"] += 1
        df_database.loc[mask,"HitPepSeq"] = df_database.loc[mask,"HitPepSeq"] + peptide + str(",")
        # print how often each peptide was found in the GroEL database
        print(peptide, found_seqs_wobbled.index(peptide), "/", len(found_seqs_wobbled), "No. of hits in the database:", len(df_database[mask]))
        total_hit_count+=len(df_database[mask])
    # print how often the peptides were found in total in the GroEL database    
    print("Total found hits over all peptides:", total_hit_count)

    #Create a new dataframe (df_HPSM) based on df_database that merges redundant sets of HitPepSeqs, counts its peptides and gives respective Acc NOs (and count of Acc NOs) of matching organisms + their taxonomic information
    df_database = df_database.drop(columns=['ProtSeq', 'TrypPeps', 'Unnamed: 0']) #removes unnecessary columns
    df_database = df_database.astype(str) #changes all columns to str, because this is necessary for the following functions
    df_database = df_database.groupby(df_database.HitPepSeq)
    df_HPSM = df_database.agg("first")
    df_HPSM.update(df_database.agg({"AccNo": ",".join, "Kingdom": ",".join, "Phylum": ",".join, "Class": ",".join, "Order": ",".join, "Family": ",".join, "Genus": ",".join})) #merges all rows with the same HitPepSeq into one row
    df_HPSM['AccNo_Count'] = df_HPSM['AccNo'].apply(lambda x: x.count(',') + 1) #give the number of AccNos (number of GroEL proteins with that HitPepSeq)
    df_HPSM['Count'] = df_HPSM['Count'].astype(int) #transforms 'Count' from str to int to sort in the next step
    df_HPSM = df_HPSM.sort_values('Count', ascending = False) #sorts the dataframe by 'Count' in descending order
    df_HPSM.reset_index(level = 0, inplace=True) #sets new index
    df_HPSM['HitPepSeq'] = df_HPSM['HitPepSeq'].str[:-1] #removes last ',' in HitPepSeq
    df_HPSM.to_csv(python_intermediate_filename, sep = '\t') #saves df_HPMS as tsv-file

    #Create a new dataframe (df_results) based on df_HPSM
    df_results = df_HPSM.iloc[:-1].copy() #removes last row (no peptide hits)
    df_results['HitPepSeq'] = df_results['HitPepSeq'].str.split(',') #transforms HitPepSeqs from str to lists
    df_results[taxonomic_columns] = df_results[taxonomic_columns].apply(lambda col: col.str.split(',')) #convert the taxonomic information about the organisms of the respective GroEL account numbers to lists to faciliate analysis
    df_results['TopRank_Count'] = (~df_results['HitPepSeq'].explode().duplicated()).groupby(level=0).sum()     #adds a new column with the count of HitPepSeq, in which each peptide only counts for the respective HitPepSeq with the highest Count (maximum parsimony). This value we call TopRank_Count   
    for col in taxonomic_columns: #create new rows with the most frequent classification on the respective taxonomic level and its frequency in decimal numbers
        df_results['Top_' + col] = df_results[col].apply(lambda row: most_frequent(row))
        df_results['Top_' + col + '_fq'] = df_results[col].apply(lambda row: most_frequent_frequency_proportion(row))


    #rearrange rows to facilitate reading
    df_results = df_results[['HitPepSeq', 'Count', 'TopRank_Count', 'AccNo', 'AccNo_Count', 'Kingdom', 'Top_Kingdom', 'Top_Kingdom_fq', 'Phylum', 'Top_Phylum', 'Top_Phylum_fq', 'Class', 'Top_Class', 'Top_Class_fq', 'Order', 'Top_Order', 'Top_Order_fq', 'Family', 'Top_Family', 'Top_Family_fq', 'Genus', 'Top_Genus', 'Top_Genus_fq']] #rearranges the dataframe to read it more easily

    #introduce strict_TopRank_Count and shared_TopRank_Count: strict_TopRank_Count counts the number of peptides that are not present in another HitPepSeq with the same amount or more peptides. shared_TopRank_Count counts the peptides that are present in another HitPepSeq with the same amount of peptides.
    unique_count_list = df_results['Count'].unique().tolist()

    df_results['HitPepSeq_of_same_count'] = ""
    df_results['new_HitPepSeq'] = ""

    HitPepSeq_list=[]
    
    #create a new column that contains all peptides that have been detected for each Count ('HitPepSeq_of_same_count') and a new column that contains all peptides that have been detected for each Count but not in for a higher Count ('new_HitPepSeq')
    for unique_count in unique_count_list:
        mask=(df_results['Count'] == unique_count)
        for each_count in df_results.loc[mask,'HitPepSeq']:
            for each_peptide in each_count:
                df_results.loc[mask,'HitPepSeq_of_same_count'] = df_results.loc[mask,'HitPepSeq_of_same_count']+each_peptide+","
                if each_peptide not in HitPepSeq_list:
                    HitPepSeq_list.append(each_peptide)
                    df_results.loc[mask,'new_HitPepSeq'] = df_results.loc[mask,'new_HitPepSeq']+each_peptide+","
    
    df_results['HitPepSeq_of_same_count'] = df_results['HitPepSeq_of_same_count'].str[:-1].str.split(',').apply(lambda row: remove_duplicates_from_list(row))
    df_results['new_HitPepSeq'] = df_results['new_HitPepSeq'].str[:-1].str.split(',').apply(lambda row: remove_duplicates_from_list(row))

    df_results['intersection_HitPepSeqs'] = [list(set(a).intersection(b)) for a, b in zip(df_results.HitPepSeq, df_results.new_HitPepSeq)]
    if total_hit_count > 0:
        df_results['len_intersection_HitPepSeqs'] = df_results['intersection_HitPepSeqs'].str.len()

    list_of_all_intersection_HitPepSeqs = list(itertools.chain.from_iterable(df_results['intersection_HitPepSeqs'].tolist()))
    shared_Top_HitPepSeqs = get_duplicates_from_list(list_of_all_intersection_HitPepSeqs)

    df_results['shared_Top_HitPepSeqs'] = df_results['intersection_HitPepSeqs'].apply (lambda row: get_intersection_from_two_lists(row, shared_Top_HitPepSeqs))
    df_results['single_Top_HitPepSeqs'] = (df_results['intersection_HitPepSeqs'].map(set) - df_results['shared_Top_HitPepSeqs'].map(set)).map(list)

    if total_hit_count > 0:
        df_results['shared_TopRank_Count'] = df_results['shared_Top_HitPepSeqs'].str.len()
        df_results['strict_TopRank_Count'] = df_results['len_intersection_HitPepSeqs']-df_results['shared_TopRank_Count']
        df_results['TopRank_Count'] = df_results['len_intersection_HitPepSeqs']
    
    if total_hit_count == 0:
        df_results['shared_TopRank_Count'] = 0
        df_results['strict_TopRank_Count'] = 0
        df_results['TopRank_Count'] = 0

    #rearrange columns to improve readability
    df_results = df_results[['HitPepSeq', 'Count', 'TopRank_Count', 'shared_Top_HitPepSeqs', 'shared_TopRank_Count', 'single_Top_HitPepSeqs', 'strict_TopRank_Count', 'AccNo', 'AccNo_Count', 'Kingdom', 'Top_Kingdom', 'Top_Kingdom_fq', 'Phylum', 'Top_Phylum', 'Top_Phylum_fq', 'Class', 'Top_Class', 'Top_Class_fq', 'Order', 'Top_Order', 'Top_Order_fq', 'Family', 'Top_Family', 'Top_Family_fq', 'Genus', 'Top_Genus', 'Top_Genus_fq']]

    #save df_results to tsv-file
    df_results.to_csv(python_output_filename, sep = '\t')
    int
    #Filter df_results to create new dataframe based on taxonomy
    print("Creating family-based output file...")
    family_based_analysis_output_filename = "C:\\Users\\klaes\\Nextcloud\\Cloud\\WP1\\Reference Data\\5ppm\\TP_biculture\\analyzed\\"+sample_name+"_family_based_min5_test.tsv"
    df_results_min5 = df_results[df_results['TopRank_Count'] > 4] #by default GroEL protein groups are filtered by a TopRank_Count >4. If you want to adjust this, change here.
    
    df_results_min5_family = df_results_min5.groupby('Top_Family').agg({'HitPepSeq': 'sum'}) #by default GroEl protein groups are merged at the family level. If you want to adjust this, change here to the level of interest.
    df_results_min5_family.reset_index(level = 0, inplace = True)

    df_results_min5_family['HitPepSeq'] = df_results_min5_family['HitPepSeq'].apply (lambda row: remove_duplicates_from_list(row))

    df_results_min5_family['non_redundant_peptide_count']= df_results_min5_family['HitPepSeq'].apply(lambda row: len(row))

    df_results_min5_family['Int'] = 0
    
    for ind in df_results_min5_family.index:
        peplist = df_results_min5_family['HitPepSeq'][ind]
        for peptide in peplist:
            for IL_peptide in make_patterns(peptide):
                if IL_peptide in MetaProSIP_Output['Peptide Sequence'].apply(lambda x: x.replace('(Carbamidomethyl)', '').replace("(Oxidation)", "")):
                    if IL_peptide not in peplist:
                        peplist.append(IL_peptide)
        mo = MetaProSIP_Output[MetaProSIP_Output['Peptide Sequence'].apply(lambda x: x.replace('(Carbamidomethyl)', '').replace("(Oxidation)", "")).isin(peplist)]
        df_results_min5_family['Int'][ind] = df_results_min5_family['Int'][ind]+sum(mo['INT 1'])
    
    df_results_min5_family
    df_results_min5_family.to_csv(family_based_analysis_output_filename, sep = '\t')

C:\Users\klaes\Nextcloud\Shared with me\Simon_Myriel_Lorenz\WP1_GroEL_method\Python\testfiles\clean_noArchaea_clean_GroEL_290780_trypsin_digests_plus_LPSN_Taxonomy_2missedcleavage_6to144aa+AK199+CV026.tsv
C:\Users\klaes\Nextcloud\Cloud\WP1\Reference Data\5ppm\TP_biculture\Galaxy179-[MetaProSIP_on_data_139,_data_22,_and_data_23__out_peptide_centric_csv].csv
['AAVEEGIVPGGGLAVIR', 'AAVEEGIVPGGGVAILR', 'AAVEEGVVPGGGVALVR', 'AEAESLYQSK', 'AETECQNTEYQQLLDIK', 'ALAAIIDLK', 'ALEEANADLEVK', 'ALEESNYELEGK', 'AMEQPLR', 'APGFGDR', 'APGFGDR', 'APVLSDSSCK', 'AQYEEIAQR', 'ATAAVVAELK', 'AVLATLEELK', 'CCTESLVNR', 'DAEAWFNEK', 'DALHATR', 'DASIQVVSAIR', 'DKFENMGAQMVK', 'DKFENMGAQMVK', 'DKFENMGAQMVK', 'DKYENMGAQMVK', 'DLLPVLEAVQK', 'DLLPVLEQINK', 'DQIVDLTVGNNK', 'DVDGAYMTK', 'DVDGAYMTK', 'DYQELMNTK', 'DYQELMNVK', 'DYQELMNTK', 'DYQELMNVK', 'EACFAVEGPK', 'EGTGDYGFNAHSETFENFFETGVIDPTK', 'EIAQVGSISANSDADIGDIIAR', 'EIAQVGSISANSDADIGDIIAR', 'EIVANAGDEASVVVNK', 'EIVANAGDEASVVVNK', 'ENTIIIDGAGEAER', 'ENTIIIDGAGEA

DLLPVLEQINK 18 / 52 No. of hits in the database: 1
EGTGDYGFNAHSETFENFFETGVIDPTK 19 / 52 No. of hits in the database: 1
EIAQVGSISANSDADIGDIIAR 20 / 52 No. of hits in the database: 5
EIVANAGDEASVVVNK 21 / 52 No. of hits in the database: 24
ENTIIIDGAGEAER 22 / 52 No. of hits in the database: 14
ENTILIDGAGEAER 23 / 52 No. of hits in the database: 3
FENMGAQMVK 24 / 52 No. of hits in the database: 55319
GDNEDQNVGIAIIR 25 / 52 No. of hits in the database: 2
GDNEDQNVGIAILR 26 / 52 No. of hits in the database: 2
GDNEDQNVGIALLR 27 / 52 No. of hits in the database: 736
GYISPYFINNPDK 28 / 52 No. of hits in the database: 30
GYLSPYFINNPDK 29 / 52 No. of hits in the database: 5503
GYLSPYFLNNPDK 30 / 52 No. of hits in the database: 3
GYISPYFITNTDK 31 / 52 No. of hits in the database: 1
GYLSPYFITNTDK 32 / 52 No. of hits in the database: 8
ISKPCTTNK 33 / 52 No. of hits in the database: 3736
LSKPCTTNK 34 / 52 No. of hits in the database: 10
MVAGINILANAVK 35 / 52 No. of hits in the database: 32
MVAGLNILAN

<ipython-input-25-123362d97ed6>:265: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_results_min5_family['Int'][ind] = df_results_min5_family['Int'][ind]+sum(mo['INT 1'])


 28 / 57 No. of hits in the contaminant database: 1
NMQDMVEDYR 29 / 57 No. of hits in the contaminant database: 1
QEYEQLIAK 30 / 57 No. of hits in the contaminant database: 1
QGVDADINGLR 31 / 57 No. of hits in the contaminant database: 1
QSLEASLAETEGR 32 / 57 No. of hits in the contaminant database: 1
QSVEADINGLR 33 / 57 No. of hits in the contaminant database: 2
QSVEADINGLRR 34 / 57 No. of hits in the contaminant database: 2
QVLDNLTMEK 35 / 57 No. of hits in the contaminant database: 1
SEIDNVKK 36 / 57 No. of hits in the contaminant database: 1
SGGGGGGGLGSGGSIR 37 / 57 No. of hits in the contaminant database: 1
SLLEGEGSSGGGGR 38 / 57 No. of hits in the contaminant database: 1
SLNNQFASFIDK 39 / 57 No. of hits in the contaminant database: 1
SLVNLGGSK 40 / 57 No. of hits in the contaminant database: 1
SQYEQLAEQNR 41 / 57 No. of hits in the contaminant database: 1
SQYEQLAEQNRK 42 / 57 No. of hits in the contaminant database: 1
STMQELNSR 43 / 57 No. of hits in the contaminant database: 1
S

KeyboardInterrupt: 